# API Example(Banking, Trading, Machine Learning...)

## mongoDB, AWS, Docker

```shell
mkdir web
mkdir db

touch web/Dockerfile
touch db/Dockerfile
cd web/

touch app.py
touch requirements.txt


```

- root /docker_compose.yml

```yml
version: '3'
services:
  web:
    build: ./web
    ports:
      - "5000:5000"
    links:
      - db
  db:
    build: ./db


```

- db/Dockerfile

```python
FROM mongo:3.6.4

```

- web/Dockerfile  

    - "./" menas "/usr/src/app"  
    
```python
FROM python:3
WORKDIR /usr/src/app
COPY requirements.txt ./  
RUN pip install --no-cache-dir -r requirements.txt
COPY . .
CMD [ "python", "app.py" ]

```

- web/requirements.txt

```python
Flask
flask_restful
pymongo
bcrypt
```

- add python library 

```python
pandas== version?
numpy== version?
matplotlib
h5py
glob3
sklearn
scikit-optimize
finance_datareader?
yfinance
pandas_datareader?
tensorflow==1.14?
keras
```


```python

# flask , flask_restful, pymongo, bcrypt library
from flask import Flask, jsonify, request
from flask_restful import Api, Resource
from pymongo import MongoClient
import bcrypt


# Create Flask object
# Create flask_restful object with Flask object input
app = Flask(__name__)
# API constructor
api = Api(app)

# Create MongoClient object
# Create new mongo db MoneyManagementDB
# Create new collections users
client = MongoClient("mongodb://db:27017")
db = client.MoneyManagementDB
users = db["Users"]

# check user name in users collections of MoneyManagementDB
def UserExist(username):
    if users.find({"Username":username}).count() == 0:
        return False
    else:
        return True


# Create Register resource
# inherit Flask restful API resource
class Register(Resource):

    # post method
    def post(self):
        # from flask request.get_json()
        #Step 1 is to get posted data by the user
        postedData = request.get_json()

        #Get the data
        username = postedData["username"]
        password = postedData["password"] #"123xyz"

        # if user exists in db, 301 status returns
        if UserExist(username):
            retJson = {
                'status':301,
                'msg': 'Invalid Username'
            }
            return jsonify(retJson)

        # password encrypted
        hashed_pw = bcrypt.hashpw(password.encode('utf8'), bcrypt.gensalt())

        # user collections inserts the new person in users collections
        #Store username and pw into the database
        users.insert({
            "Username": username,
            "Password": hashed_pw,
            "Own":0,
            "Debt":0
        })

        retJson = {
            "status": 200,
            "msg": "You successfully signed up for the API"
        }
        return jsonify(retJson)
    
```

- helper function 

- DB columns: 

```
    "Username"
    "Password"
    "Own"
    "Debt"
```

```python
# verify password , compare hashed_pw from db and password from json
def verifyPw(username, password):
    if not UserExist(username):
        return False

    hashed_pw = users.find({
        "Username":username
    })[0]["Password"]

    if bcrypt.hashpw(password.encode('utf8'), hashed_pw) == hashed_pw:
        return True
    else:
        return False

# return user's cash in user's account
def cashWithUser(username):
    cash = users.find({
        "Username":username
    })[0]["Own"]
    return cash

# return debt from user['Username'][0]['Debt']
def debtWithUser(username):
    debt = users.find({
        "Username":username
    })[0]["Debt"]
    return debt

def generateReturnDictionary(status, msg):
    retJson = {
        "status": status,
        "msg": msg
    }
    return retJson

# return Error msg,True/False
def verifyCredentials(username, password):
    if not UserExist(username):
        return generateReturnDictionary(301, "Invalid Username"), True

    correct_pw = verifyPw(username, password)

    if not correct_pw:
        return generateReturnDictionary(302, "Incorrect Password"), True

    return None, False



# mongodb collection_name.update method
# update mongo db Own field in user collections
def updateAccount(username, balance):
    users.update({"Username": username},{ "$set":{"Own": balance} } )
    
    

```



- ADD flask resource API

```python


```